# Revisão - Image Classification

Revisão focada em aprender a sintaxe e conceitos passados nessa pasta

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
import torch.nn.functional as F
import torchvision
from torchvision import transforms
from PIL import Image

## Dataset e DataLoader 

Quando usamos **PyTorch**, precisamos preparar os dados antes de treinar uma rede neural. Isso pode ser demorado, mas PyTorch tem algumas ferramentas que facilitam esse processo:  

1. **Dataset** → É uma "caixa" que guarda os dados. Podemos acessar qualquer dado individualmente dentro dela.  
2. **Data Loader** → Pega os dados do Dataset e manda para a rede neural. Ele pode definir quantos dados serão enviados por vez e quantos processos vão trabalhar ao mesmo tempo.  

---

### Criando um Dataset  
Para que o PyTorch consiga trabalhar com qualquer tipo de dado (imagem, áudio, texto, etc.), ele precisa seguir um padrão simples:  

- Ter um método que retorna o **tamanho do dataset**.  
- Ter um método que retorna **um item específico** do dataset (como uma imagem e seu rótulo).  

Isso é útil, mas pode ser trabalhoso. Então, PyTorch já oferece ferramentas prontas!  

---

### **Torchvision**   
Se estamos lidando com **imagens**, o PyTorch tem o `ImageFolder`, que organiza as imagens automaticamente. Basta colocar as imagens em pastas com o nome da categoria. Exemplo:  

```
/train  
 ├── cat/ (todas as imagens de gatos)  
 ├── fish/ (todas as imagens de peixes)  
```

In [ ]:
import torchvision
from torchvision import transforms

train_data_path = "./train/"

transforms = transforms.Compose([
    transforms.Resize(64),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
std=[0.229, 0.224, 0.225])
])

train_data = torchvision.datasets.ImageFolder(root=train_data_path, transform = transforms)

### **Por que fazemos essas transformações?**  
**Redimensionamento** → Para que todas as imagens tenham o mesmo tamanho (64x64) e o processamento seja mais rápido.  
**Conversão para Tensor** → Para que o PyTorch possa trabalhar com os dados.  
**Normalização** → Para manter os valores entre 0 e 1, evitando que fiquem muito grandes e atrapalhem o aprendizado da rede neural.  

In [ ]:
val_data_path = "./val/"
val_data = torchvision.datasets.ImageFolder(root=val_data_path, transform = transforms)

test_data_path = "./test/"
test_data = torchvision.datasets.ImageFolder(root=test_data_path, transform=transforms)


### Conjunto de Treinamento  
Usado na fase de treinamento para atualizar o modelo.  

### Conjunto de Validação  
Usado para avaliar como o modelo está generalizando para o domínio do problema, em vez de apenas se ajustar aos dados de treinamento; não é usado para atualizar o modelo diretamente.  

### Conjunto de Teste  
Um conjunto de dados final que fornece uma avaliação definitiva do desempenho do modelo após a conclusão do treinamento.  


In [ ]:
batch_size = 64

train_data_loader = data.DataLoader(train_data, batch_size)
val_data_loader = data.DataLoader(val_data, batch_size=batch_size)
test_data_loader = data.DataLoader(test_data, batch_size=batch_size)

## Rede Neural

In [ ]:
class SimpleNet(nn.Module):
    def __init__(self):
        super(Net, self).__init__
        self.fc1 = nn.Linear(12288, 84)
        self.fc2 = nn.Linear(84, 50)
        self.fc3 = nn.Linear(50, 2)

    def forward(self):
        x = x.view(-1, 12288) # achata imagem para um vetor 1D
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x) # a crossentropy ja vai aplicar o softmax
        return x
    
simplenet = SimpleNet()

## Funções de Perda no PyTorch

O PyTorch já vem com várias funções de perda prontas para diferentes tipos de problemas. Nesse caso estamos lidando com classificação multiclasse, então usamos CrossEntropyLoss

O CrossEntropyLoss já aplica a função Softmax

In [ ]:
criterion = nn.CrossEntropyLoss()

## Otimizador

Um otimizador ajusta os pesos da rede para que a função de perda seja minimizada. Ele faz isso calculando gradientes e alterando os pesos na direção certa

In [ ]:

optimizer = optim.Adam(simplenet.parameters(), lr=0.001) # passa os pesos da rede neural

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda") 
else:
    device = torch.device("cpu")

simplenet.to(device)

## Treinamento

In [ ]:
for data, label in train_loader:
    output = simplenet(data)              # passa os dados pela rede
    loss = criterion(output, label)       # calcula a perda
    optimizer.zero_grad()                 # zera os gradientes anteriores
    loss.backward()                       # calcula os gradientes
    optimizer.step()  

 Ou fazer de forma mais robusta

In [ ]:
def train(model, optimizer, loss_fn, train_loader, val_loader, epochs=20, device="cpu"):
    for epoch in range(epochs):
        training_loss = 0.0
        valid_loss = 0.0
        
        model.train()
        for batch in train_loader:
            optimizer.zero_grad()
            inputs, targets = batch
            inputs, targets = inputs.to(device), targets.to(device)
            output = model(inputs)
            loss = loss_fn(output, targets)
            loss.backward()
            optimizer.step()
            training_loss += loss.data.item()
        
        training_loss /= len(train_loader)
        
        model.eval()
        num_correct = 0
        num_examples = 0
        with torch.no_grad():
            for batch in val_loader:
                inputs, targets = batch
                inputs, targets = inputs.to(device), targets.to(device)
                output = model(inputs)
                loss = loss_fn(output, targets)
                valid_loss += loss.data.item()
                
                correct = torch.eq(torch.max(F.softmax(output, dim=1), dim=1)[1], targets).view(-1)
                num_correct += torch.sum(correct).item()
                num_examples += correct.shape[0]
        
        valid_loss /= len(val_loader)
        accuracy = num_correct / num_examples
        
        print(f'Epoch: {epoch}, Training Loss: {training_loss:.2f}, Validation Loss: {valid_loss:.2f}, Accuracy: {accuracy:.2f}')

In [ ]:
train(simplenet, optimizer, torch.nn.CrossEntropyLoss(), train_data_loader, test_data_loader, device)

## Previsões

In [ ]:
to_tensor = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor()
])

labels = ['cat', 'fish']
img = Image.open(FILENAME)
img = to_tensor(img)
img = img.unsqueeze(0)  # adiciona uma dimensão extra para criar um "batch" de tamanho 1

prediction = simplenet(img)
prediction = prediction.argmax()
print(labels[prediction])